In [111]:
import csv
import sys
import pandas as pd
import numpy as np
from operator import itemgetter
dataPath = "./INTEGRATED-DATASET.csv"



dataSet = pd.read_csv(dataPath)
dataSet.columns = ["Name", "Borough", "Type", "InspectionSeason", "Reason", "Critical", "Score", "Grade", "GradeSeason"]
dataSet
min_supp = 0.4*len(dataSet)
min_conf = 0.1*len(dataSet)



In [103]:
def get_L1(dataSet, one_item):
    for item in one_item:
        cntType = dataSet.groupby(item).count()
        one_item[item] = [x for x in one_item[item] if cntType.at[x, 'Name'] >= min_supp]

def get_Lk(one_item):
    Union_Lk = []
    Uninion_Lk.append(one_item)
    while len(Union_Lk[-1])!=0:
        Ck = apriori_gen(Uniion_Lk[-1])
        Ck.sort()
        Union_Lk.append([])

        
def apriori_gen(Lk_minus1):
    Ck = []
    for i in xrange(0, len(Lk_minus1)):
        for j in xrange(i+1, len(Lk_minus1)):
            if check_match(Lk_minus1[i], Lk_minus1[j]):
            
            
        
def check_match(set1, set2):
    for i in xrange(len(set1)-1):
        if(set1[i]!=set2[j]) return False
    if set

In [115]:
def init_one_item(one_item, key, values):
    for val in values:
        one_item.append((key, val))

boroughs = dataSet["Borough"].unique()
grades = dataSet["Grade"].unique()
types = dataSet["Type"].unique()
gradeSeason = dataSet["GradeSeason"].unique()

gradeSeason = np.delete(gradeSeason, gradeSeason.shape[0]-1).tolist()
types = np.delete(types, types.shape[0]-1).tolist()
grades = np.delete(grades, grades.shape[0]-1).tolist()
boroughs = np.delete(boroughs, boroughs.shape[0]-1).tolist()

one_item=[]
init_one_item(one_item, "Type", types)
init_one_item(one_item, "Borough", boroughs)
init_one_item(one_item, "Grades", grades)
init_one_item(one_item, "GradeSeason", gradeSeason)

one_item = sorted(one_item, key=itemgetter(0))
print(one_item)

get_L1(dataSet, one_item)
# print (one_item)

[('Borough', 'BRONX'), ('Borough', 'MANHATTAN'), ('Borough', 'BROOKLYN'), ('Borough', 'QUEENS'), ('Borough', 'STATEN ISLAND'), ('GradeSeason', 'Winter-GRADE DATE'), ('GradeSeason', 'Summer-GRADE DATE'), ('GradeSeason', 'Spring-GRADE DATE'), ('Grades', 'A'), ('Grades', 'B'), ('Grades', 'C'), ('Grades', 'P'), ('Grades', 'Z'), ('Type', 'NorthAmerican'), ('Type', 'European'), ('Type', 'Asian'), ('Type', 'Other'), ('Type', 'African'), ('Type', 'SouthAmerican')]
